# AEP Storm Analysis with RAS-Commander

This notebook automates the end-to-end process of analyzing multiple storm events with different Annual Exceedance Probabilities (AEP) in HEC-RAS. It covers:

1. Generating hyetographs from NOAA Atlas 14 precipitation data
2. Creating HEC-RAS plan files for each AEP event
3. Creating unsteady flow files with the generated hyetographs
4. Executing multiple plans in parallel
5. Analyzing and visualizing the results

This automation is particularly useful for analyzing how a drainage system performs under different storm frequencies, from common events (e.g., 2-year) to rare events (e.g., 100-year).

## Setup and Import Libraries

First, we'll import all the necessary libraries and set up our environment.

In [1]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
from pathlib import Path
from datetime import datetime
from IPython import display
import psutil  # For getting system CPU info

# Install ras-commander if not already installed
# Uncomment this line if you need to install the package
# !pip install ras-commander

# Import RAS-Commander modules
from ras_commander import *

In [ ]:
RasExamples.extract_project(["Davis"])
# This loads the project in fresh

## Define Hyetograph Generation Functions

These functions handle reading precipitation frequency data from NOAA Atlas 14 and generating balanced storm hyetographs using the Alternating Block Method.

In [3]:
def parse_duration(duration_str):
    """
    Parses a duration string and converts it to hours.
    Examples: "5-min:" -> 0.0833 hours, "2-hr:" -> 2 hours, "2-day:" -> 48 hours
    """
    match = re.match(r'(\d+)-(\w+):', duration_str.strip())
    if not match:
        raise ValueError(f"Invalid duration format: {duration_str}")
    value, unit = match.groups()
    value = int(value)
    unit = unit.lower()
    if unit in ['min', 'minute', 'minutes']:
        hours = value / 60.0
    elif unit in ['hr', 'hour', 'hours']:
        hours = value
    elif unit in ['day', 'days']:
        hours = value * 24
    else:
        raise ValueError(f"Unknown time unit in duration: {unit}")
    return hours

def read_precipitation_data(csv_file):
    """
    Reads the precipitation frequency CSV and returns a DataFrame
    with durations in hours as the index and ARIs as columns.
    """
    with open(csv_file, 'r') as f:
        lines = f.readlines()

    header_line_idx = None
    header_pattern = re.compile(r'^by duration for ari', re.IGNORECASE)

    # Locate the header line
    for idx, line in enumerate(lines):
        if header_pattern.match(line.strip().lower()):
            header_line_idx = idx
            break

    if header_line_idx is None:
        raise ValueError('Header line for precipitation frequency estimates not found in CSV file.')

    # Extract the ARI headers from the header line
    header_line = lines[header_line_idx].strip()
    headers = [item.strip() for item in header_line.split(',')]
    
    if len(headers) < 2:
        raise ValueError('Insufficient number of ARI columns found in the header line.')

    aris = headers[1:]  # Exclude the first column which is the duration

    # Define the pattern for data lines (e.g., "5-min:", "10-min:", etc.)
    duration_pattern = re.compile(r'^\d+-(min|hr|day):')

    # Initialize lists to store durations and corresponding depths
    durations = []
    depths = {ari: [] for ari in aris}

    # Iterate over the lines following the header to extract data
    for line in lines[header_line_idx + 1:]:
        line = line.strip()
        if not line:
            continue  # Skip empty lines
        if not duration_pattern.match(line):
            break  # Stop if the line does not match the duration pattern
        parts = [part.strip() for part in line.split(',')]
        if len(parts) != len(headers):
            raise ValueError(f"Data row does not match header columns: {line}")
        duration_str = parts[0]
        try:
            duration_hours = parse_duration(duration_str)
        except ValueError as ve:
            print(f"Skipping line due to error: {ve}")
            continue  # Skip lines with invalid duration formats
        durations.append(duration_hours)
        for ari, depth_str in zip(aris, parts[1:]):
            try:
                depth = float(depth_str)
            except ValueError:
                depth = np.nan  # Assign NaN for invalid depth values
            depths[ari].append(depth)

    # Create the DataFrame
    df = pd.DataFrame(depths, index=durations)
    df.index.name = 'Duration_hours'

    # Drop any rows with NaN values
    df = df.dropna()

    return df

def interpolate_depths(df, total_duration):
    """
    Interpolates precipitation depths for each ARI on a log-log scale
    for each hour up to the total storm duration.
    """
    T = total_duration
    t_hours = np.arange(1, T+1)
    D = {}
    for ari in df.columns:
        durations = df.index.values
        depths = df[ari].values
        # Ensure all depths are positive
        if np.any(depths <= 0):
            raise ValueError(f"Non-positive depth value in ARI {ari}")
        # Log-log interpolation
        log_durations = np.log(durations)
        log_depths = np.log(depths)
        log_t = np.log(t_hours)
        log_D_t = np.interp(log_t, log_durations, log_depths)
        D_t = np.exp(log_D_t)
        D[ari] = D_t
    return D

def compute_incremental_depths(D, total_duration):
    """
    Computes incremental precipitation depths for each hour.
    I(t) = D(t) - D(t-1), with D(0) = 0.
    """
    incremental_depths = {}
    for ari, D_t in D.items():
        I_t = np.empty(total_duration)
        I_t[0] = D_t[0]  # I(1) = D(1) - D(0) = D(1)
        I_t[1:] = D_t[1:] - D_t[:-1]
        incremental_depths[ari] = I_t
    return incremental_depths

def assign_alternating_block(sorted_depths, max_depth, central_index, T):
    """
    Assigns incremental depths to the hyetograph using the Alternating Block Method.
    """
    hyetograph = [0.0] * T
    hyetograph[central_index] = max_depth
    remaining_depths = sorted_depths.copy()
    remaining_depths.remove(max_depth)
    left = central_index - 1
    right = central_index + 1
    toggle = True  # Start assigning to the right
    for depth in remaining_depths:
        if toggle and right < T:
            hyetograph[right] = depth
            right += 1
        elif not toggle and left >= 0:
            hyetograph[left] = depth
            left -= 1
        elif right < T:
            hyetograph[right] = depth
            right += 1
        elif left >= 0:
            hyetograph[left] = depth
            left -= 1
        else:
            print("Warning: Not all incremental depths assigned.")
            break
        toggle = not toggle
    return hyetograph

def generate_hyetograph(incremental_depths, position_percent, T):
    """
    Generates the hyetograph for a given ARI using the Alternating Block Method.
    """
    max_depth = np.max(incremental_depths)
    incremental_depths_list = incremental_depths.tolist()
    central_index = int(round(T * position_percent / 100)) - 1
    central_index = max(0, min(central_index, T - 1))
    sorted_depths = sorted(incremental_depths_list, reverse=True)
    hyetograph = assign_alternating_block(sorted_depths, max_depth, central_index, T)
    return hyetograph

def save_hyetograph(hyetograph, ari, output_dir, position_percent, total_duration):
    """
    Saves the hyetograph to a CSV file.
    """
    df = pd.DataFrame({
        'Time_hour': np.arange(1, total_duration + 1),
        'Precipitation_in': hyetograph
    })
    filename = f'hyetograph_ARI_{ari}_years_pos{position_percent}pct_{total_duration}hr.csv'
    output_file = os.path.join(output_dir, filename)
    df.to_csv(output_file, index=False)
    print(f"Hyetograph for ARI {ari} years saved to {output_file}")
    return output_file

def plot_multiple_hyetographs(aris, position_percent, total_duration, output_dir='hyetographs'):
    """
    Plots multiple hyetographs for specified ARIs on the same figure for comparison.
    """
    plt.figure(figsize=(14, 7))
    
    for ari in aris:
        # Ensure ARI is a string for consistent filename formatting
        ari_str = str(ari)
        
        # Construct the filename based on the naming convention
        filename = f'hyetograph_ARI_{ari_str}_years_pos{position_percent}pct_{total_duration}hr.csv'
        filepath = os.path.join(output_dir, filename)
        
        # Check if the file exists
        if not os.path.exists(filepath):
            print(f"Warning: File '{filename}' does not exist in the directory '{output_dir}'. Skipping this ARI.")
            continue
        
        # Read the hyetograph data
        try:
            hyetograph_df = pd.read_csv(filepath)
            print(f"Successfully read the hyetograph data from '{filename}'.")
        except Exception as e:
            print(f"Error reading the hyetograph CSV file '{filename}': {e}")
            continue
        
        # Plot the hyetograph
        plt.bar(hyetograph_df['Time_hour'], hyetograph_df['Precipitation_in'], 
                width=0.8, edgecolor='black', alpha=0.5, label=f'ARI {ari_str} years')
    
    # Customize the plot
    plt.xlabel('Time (Hour)', fontsize=14)
    plt.ylabel('Incremental Precipitation (inches)', fontsize=14)
    plt.title(f'Comparison of Hyetographs for ARIs {aris}\nPosition: {position_percent}% | Duration: {total_duration} Hours', fontsize=16)
    plt.legend()
    plt.xticks(range(1, total_duration + 1, max(1, total_duration // 24)))  # Adjust x-ticks based on duration
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

## Generate AEP Hydrographs

This cell orchestrates the entire AEP analysis process, generating hyetographs

In [ ]:

"""
Main function to run the entire AEP analysis process.
"""
# Set the paths and parameters
input_csv = 'data/PF_Depth_English_PDS_DavisCA.csv'  # Path to NOAA Atlas 14 data
output_dir = 'hyetographs'  # Directory for saving hyetographs
position_percent = 50  # Position percentage for the maximum incremental depth block
total_duration = 24  # Storm duration in hours
base_plan = "02"  # Base plan to clone

# Set the AEP events (return periods in years)
aep_events = [2, 5, 10, 25, 50, 100]

# Ensure the output directory exists
Path(output_dir).mkdir(parents=True, exist_ok=True)
print(f"Output directory is set to: {output_dir}")

#-------------------------------------------------------------------------
# Step 1: Generate hyetographs for each AEP event
#-------------------------------------------------------------------------
print("\nStep 1: Generating hyetographs for each AEP event...")

try:
    # Read precipitation data
    df = read_precipitation_data(input_csv)
    print("Successfully read the input CSV file.")
    
    # Display the first few rows of the DataFrame to verify
    print("\nPrecipitation Frequency Data from Atlas 14:")
    display.display(df.head())
    
    # Interpolate depths
    D = interpolate_depths(df, total_duration)
    print("Successfully interpolated precipitation depths.")

    print("Array D with interpolated depths")
    display.display(D)
    
    # Compute incremental depths
    inc_depths = compute_incremental_depths(D, total_duration)
    print("Successfully computed incremental depths.")
    
    # Show Incremental Depths
    print("Array inc_depths Contents ")
    display.display(inc_depths)

    # Generate and save hyetographs for each AEP
    hyetograph_files = {}
    for ari in aep_events:
        ari_str = str(ari)
        if ari_str in inc_depths:
            hyetograph = generate_hyetograph(inc_depths[ari_str], position_percent, total_duration)
            file_path = save_hyetograph(hyetograph, ari_str, output_dir, position_percent, total_duration)
            hyetograph_files[ari_str] = file_path
        else:
            print(f"Warning: ARI {ari_str} not found in the data. Skipping.")
    
    print("\nAll hyetographs have been generated and saved.")
    
    # Plot the hyetographs for comparison
    plot_multiple_hyetographs(aep_events, position_percent, total_duration, output_dir)
    
except Exception as e:
    print(f"Error generating hyetographs: {e}")
    raise

In [ ]:
#-------------------------------------------------------------------------
# Initialize the HEC-RAS project
#-------------------------------------------------------------------------
print("\nStep 2: Initializing the HEC-RAS ras...")

# Define the path to the Davis project
current_dir = Path.cwd()
pipes_ex_path = current_dir / "example_projects" / "Davis"

# Check if the project exists
if not pipes_ex_path.exists():
    # Extract the project if needed
    RasExamples.extract_project(["Davis"])

# Initialize the RAS project
init_ras_project(pipes_ex_path, "6.6")
print(f"Initialized HEC-RAS project: {ras.project_name}")

# Display the existing plans
print("\nExisting plans in the project:")
display.display(ras.plan_df)

In [ ]:
ras.plan_df

In [ ]:
ras.unsteady_df

In [ ]:
print(ras.boundaries_df)

In [ ]:
# Get the path to unsteady flow file associated with Plan "01"
unsteady_file = RasPlan.get_unsteady_path("01")
print(f"Unsteady flow file path: {unsteady_file}")

In [ ]:
unsteady_file

In [ ]:
# Extract boundary conditions and tables
boundaries_df = RasUnsteady.extract_boundary_and_tables(unsteady_file)
print(f"Extracted {len(boundaries_df)} boundary conditions from the unsteady flow file.")

In [ ]:
boundaries_df

In [ ]:
# Display the contents of Unsteady File
with open(unsteady_file, 'r') as f:
    unsteady_contents = f.read()
print(f"Contents of unsteady flow file {unsteady_file}:")
print("-" * 80)
print(unsteady_contents)
print("-" * 80)

### To implement AEP event hydrographs, we will edit the Precipitation Hydrograph table
We will need to edit both the number of values, as well as replacing the existing fixed-width table.  

#### Define HEC-RAS Plan and Unsteady Flow File Functions

These functions handle creating HEC-RAS plan files and unsteady flow files for each AEP event. They apply the generated hyetographs to the boundary conditions.

In [14]:
def create_plan_for_aep(base_plan, aep_years, duration_hours, hyetograph_file, project):
    """
    Creates a new plan for a specific AEP event.
    """
    # Create plan name and short ID
    plan_name = f"{aep_years}YR-{duration_hours}HR"
    
    print(f"Creating new plan '{plan_name}'...")
    
    # Clone the base plan
    new_plan_number = RasPlan.clone_plan(base_plan, new_plan_shortid=plan_name, ras_object=project)
    print(f"Created new plan: {new_plan_number}")
    
    # Clone the unsteady flow file from the base plan
    base_unsteady = None
    for _, row in project.plan_df.iterrows():
        if row['plan_number'] == base_plan:
            base_unsteady = row.get('unsteady_number', None)
            
    if base_unsteady is None:
        raise ValueError(f"Could not find unsteady flow file for base plan {base_plan}")

    
    new_unsteady_number = RasPlan.clone_unsteady(base_unsteady, ras_object=project)
    print(f"Created new unsteady flow file: {new_unsteady_number}")
    
    # Update the unsteady flow file with the hyetograph data
    unsteady_file_path = RasPlan.get_unsteady_path(new_unsteady_number, ras_object=project)
    
    
    # Update the flow title to reflect the AEP event
    new_title = f"{aep_years}YR-{duration_hours}HR Storm"
    RasUnsteady.update_flow_title(unsteady_file_path, new_title, ras_object=project)
    print(f"Updated unsteady flow title to: {new_title}")
    
    # Modify the unsteady flow file with the hyetograph data
    success = modify_unsteady_flow_with_hyetograph(unsteady_file_path, hyetograph_file, project)
    if success:
        print(f"Successfully applied hyetograph data from {hyetograph_file} to unsteady flow file")
    else:
        print(f"Warning: Failed to apply hyetograph data. Unsteady flow file may need manual modification.")
    
    # Assign the unsteady flow file to the plan
    RasPlan.set_unsteady(new_plan_number, new_unsteady_number, ras_object=project)
    print(f"Assigned unsteady flow file {new_unsteady_number} to plan {new_plan_number}")
    '''
    # Update the plan description
    description = f"AEP {aep_years}-year, {duration_hours}-hour storm\n"
    description += f"Created: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
    description += f"Based on plan {base_plan}\n"
    description += f"Hyetograph from: {os.path.basename(hyetograph_file)}"
    
    RasPlan.update_plan_description(new_plan_number, description, ras_object=project)
    print(f"Updated plan description for plan {new_plan_number}")
    
    return new_plan_number, new_unsteady_number
    '''

In [15]:
def modify_unsteady_flow_with_hyetograph(unsteady_file_path, hyetograph_file, project):
    """
    Modifies an unsteady flow file to incorporate hyetograph data as precipitation.
    
    Parameters:
    - unsteady_file_path: Path to the unsteady flow file
    - hyetograph_file: Path to the hyetograph data CSV
    - project: RAS project object
    
    Returns:
    - Boolean indicating success
    """
    try:
        # Read the hyetograph data
        hyetograph_df = pd.read_csv(hyetograph_file)
        print(f"Loaded hyetograph from {hyetograph_file} with {len(hyetograph_df)} values")
        
        # Read the unsteady flow file
        with open(unsteady_file_path, 'r') as file:
            lines = file.readlines()
        
        # Find the sections that need to be modified
        precip_hydrograph_index = None
        
        for i, line in enumerate(lines):
            if line.startswith("Precipitation Hydrograph="):
                precip_hydrograph_index = i
                break
        
        if precip_hydrograph_index is None:
            print("Cannot find Precipitation Hydrograph section in unsteady file.")
            return False
        
        # Get the time interval from the hyetograph
        time_interval = "1HOUR"  # Default
        if "Time_hour" in hyetograph_df.columns and len(hyetograph_df) > 1:
            hour_diff = hyetograph_df["Time_hour"].iloc[1] - hyetograph_df["Time_hour"].iloc[0]
            time_interval = f"{int(hour_diff)}HOUR" if hour_diff >= 1 else f"{int(hour_diff*60)}MIN"
        
        # Format the precipitation values for the hydrograph
        precipitation_values = hyetograph_df["Precipitation_in"].values
        
        # Create the Precipitation Hydrograph line
        precip_line = f"Precipitation Hydrograph= {len(precipitation_values)} \n"
        
        # Format the values in groups of 10 per line
        value_lines = []
        for i in range(0, len(precipitation_values), 10):
            row_values = precipitation_values[i:i+10]
            row_line = "".join([f"{value:8.2f}" for value in row_values]) + "\n"
            value_lines.append(row_line)
            
        # Remove old hydrograph data - find end of current hydrograph
        current_line = precip_hydrograph_index + 1
        while current_line < len(lines) and not any(lines[current_line].startswith(prefix) for prefix in ["DSS Path=", "Use DSS=", "Use Fixed Start Time="]):
            current_line += 1
            
        # Replace the hydrograph section
        lines[precip_hydrograph_index:current_line] = [precip_line] + value_lines
            
        # Write the modified file back
        with open(unsteady_file_path, 'w') as file:
            file.writelines(lines)
            
        print(f"Successfully applied hyetograph data from {hyetograph_file} to unsteady flow file.")
        print(f"Added {len(precipitation_values)} precipitation values with interval {time_interval}")
        return True
        
    except Exception as e:
        print(f"Error modifying unsteady flow file: {str(e)}")
        import traceback
        traceback.print_exc()
        return False

In [ ]:
#-------------------------------------------------------------------------
# Create new plans for each AEP event
#-------------------------------------------------------------------------
print("\nStep 3: Creating new plans for each AEP event...")

new_plan_numbers = []

for ari in aep_events:
    ari_str = str(ari)
    if ari_str in hyetograph_files:
        try:
            # Create a new plan for this AEP event
            new_plan_number, _ = create_plan_for_aep(
                base_plan=base_plan,
                aep_years=ari_str,
                duration_hours=total_duration,
                hyetograph_file=hyetograph_files[ari_str],
                project=ras
            )
            new_plan_numbers.append(new_plan_number)
        except Exception as e:
            print(f"Error creating plan for AEP {ari_str}: {e}")

# Display the updated plans
print("\nUpdated plans in the project:")
display.display(ras.plan_df)

In [ ]:
# Get the path to unsteady flow file associated with Plan "03"
unsteady_file_rev = RasPlan.get_unsteady_path("03")
print(f"Unsteady flow file path: {unsteady_file_rev}")

In [ ]:
unsteady_file_rev

In [ ]:
with open(unsteady_file_rev, 'r') as f:
    unsteady_contents_rev = f.read()
print(f"Contents of unsteady flow file for plan 03 ({unsteady_file_rev}):")
print("-" * 80)
print(unsteady_contents_rev)
print("-" * 80)

In [ ]:
import difflib

def show_file_diff(file1_path, file2_path, context_lines=3):
    """
    Shows the differences between two files with context.
    
    Parameters:
    - file1_path: Path to the first file
    - file2_path: Path to the second file
    - context_lines: Number of context lines to show around differences
    """
    # Read the file contents
    with open(file1_path, 'r') as file1:
        file1_lines = file1.readlines()
    
    with open(file2_path, 'r') as file2:
        file2_lines = file2.readlines()
    
    # Create a differ object
    differ = difflib.unified_diff(
        file1_lines, 
        file2_lines,
        fromfile=str(file1_path),
        tofile=str(file2_path),
        n=context_lines
    )
    
    # Convert differ output to a string
    diff_text = ''.join(differ)
    
    # If no differences found
    if not diff_text:
        print(f"No differences found between {file1_path} and {file2_path}")
        return
    
    # Print the differences
    print(f"Differences between {file1_path} and {file2_path}:")
    print("-" * 80)
    print(diff_text)
    print("-" * 80)
    
    # Count added, removed, and modified lines
    added = sum(1 for line in diff_text.splitlines() if line.startswith('+') and not line.startswith('+++'))
    removed = sum(1 for line in diff_text.splitlines() if line.startswith('-') and not line.startswith('---'))
    
    print(f"Summary: {added} additions, {removed} removals")

# Show differences between the unsteady flow files
if 'unsteady_file' in locals() and 'unsteady_file_rev' in locals():
    show_file_diff(unsteady_file, unsteady_file_rev)
else:
    print("Error: One or both unsteady flow file variables not defined.")
    print("Please run the cells that define unsteady_file and unsteady_file_rev first.")

#### Define Parallel Execution and Results Analysis Functions

These functions manage parallel plan execution with resource optimization and extract, analyze, and visualize the results.

In [21]:
def execute_plans_in_parallel(plan_numbers, project, max_workers=None, cores_per_worker=2):
    """
    Executes multiple plans in parallel.
    """
    # Calculate optimal number of workers if not provided
    if max_workers is None:
        physical_cores = psutil.cpu_count(logical=False)  # Physical cores only
        max_workers = max(1, physical_cores // cores_per_worker)
    
    print(f"Executing {len(plan_numbers)} plans in parallel with {max_workers} workers, " + 
          f"each using {cores_per_worker} cores...")
    
    # Create compute folder
    compute_folder = Path(project.project_folder) / "compute_aep_parallel"
    compute_folder.mkdir(parents=True, exist_ok=True)
    
    # Execute plans in parallel
    start_time = time.time()
    
    results = RasCmdr.compute_parallel(
        plan_number=plan_numbers,
        max_workers=max_workers,
        num_cores=cores_per_worker,
        dest_folder=compute_folder,
        clear_geompre=True,
        overwrite_dest=True,
        ras_object=project
    )
    
    end_time = time.time()
    total_duration = end_time - start_time
    
    print(f"Parallel execution completed in {total_duration:.2f} seconds")
    
    # Create a DataFrame from the execution results
    results_df = pd.DataFrame([
        {"Plan": plan, "Success": success}
        for plan, success in results.items()
    ])
    
    # Sort by plan number
    results_df = results_df.sort_values("Plan")
    
    print("\nExecution Results:")
    display.display(results_df)
    
    return results, compute_folder

In [ ]:
# Get the path to unsteady flow file associated with Plan "01"
unsteady_file = RasPlan.get_unsteady_path("01")
print(f"Unsteady flow file path: {unsteady_file}")

In [23]:
#-------------------------------------------------------------------------
# Execute all plans in parallel
#-------------------------------------------------------------------------

In [ ]:
# Set computation parameters for better performance
for plan_number in new_plan_numbers:
    RasPlan.set_num_cores(plan_number, 2, ras_object=ras)
    RasPlan.update_plan_intervals(
        plan_number,
        computation_interval="15MIN",
        output_interval="30MIN",
        mapping_interval="1HOUR",
        ras_object=ras
    )
    print(f"Updated computation settings for plan {plan_number}")

# Execute plans in parallel
results, compute_folder = execute_plans_in_parallel(
    plan_numbers=new_plan_numbers,
    project=ras
)

print("\nStep 4: Executing all plans in parallel...")

results, compute_folder = execute_plans_in_parallel(
    plan_numbers=new_plan_numbers,
    project=ras
)

In [ ]:
ras.plan_df

In [ ]:
runtime_df = HdfResultsPlan.get_runtime_data("02")

In [ ]:
runtime_df

In [ ]:
pipe_flow_ds = HdfPipe.get_pipe_network_timeseries("02", variable="Pipes/Pipe Flow DS")


In [ ]:
pipe_flow_ds 

In [ ]:
node_ws = HdfPipe.get_pipe_network_timeseries("02", variable="Nodes/Water Surface")

In [ ]:
node_ws

In [32]:
def analyze_results(results, compute_folder, project):
    """
    Analyzes the results from multiple plans.
    """
    print("Analyzing results from parallel execution...")
    
    # Initialize a RAS project in the compute folder
    compute_project = RasPrj()
    compute_project = init_ras_project(compute_folder, "6.6", ras_object=compute_project)
    print(f"Initialized compute project: {compute_project.project_name}")
    
    # Check which plans have results
    plans_with_results = compute_project.plan_df[compute_project.plan_df['HDF_Results_Path'].notna()]
    print(f"\nFound {len(plans_with_results)} plans with results:")
    display.display(plans_with_results[['plan_number', 'Short Identifier', 'HDF_Results_Path']])
    
    # Initialize a dictionary to store analysis results
    analysis_results = {}
    
    # Analyze each plan's results
    for idx, row in plans_with_results.iterrows():
        plan_number = row['plan_number']
        plan_name = row['Short Identifier']
        hdf_path = row['HDF_Results_Path']
        
        print(f"\nAnalyzing results for plan {plan_number} ({plan_name})...")
        
        try:
            # Get runtime data
            runtime_df = HdfResultsPlan.get_runtime_data(hdf_path)
            
            if runtime_df is not None and not runtime_df.empty:
                # Extract key metrics
                sim_duration = runtime_df['Simulation Duration (s)'].iloc[0]
                compute_time = runtime_df['Complete Process (hr)'].iloc[0]
                compute_speed = runtime_df['Complete Process Speed (hr/hr)'].iloc[0]
                
                # Get pipe network results
                try:
                    # Get pipe flow data
                    pipe_flow_ds = HdfPipe.get_pipe_network_timeseries(hdf_path, variable="Pipes/Pipe Flow DS")
                    node_ws = HdfPipe.get_pipe_network_timeseries(hdf_path, variable="Nodes/Water Surface")
                    
                    # Convert xarray DataArrays to numpy arrays and compute statistics
                    pipe_flow_array = pipe_flow_ds.values
                    node_ws_array = node_ws.values
                    
                    # Calculate maximum flows and water surfaces
                    max_flows = np.nanmax(pipe_flow_array, axis=0)  # Max over time for each location
                    avg_max_flow = np.nanmean(max_flows)
                    max_max_flow = np.nanmax(max_flows)
                    
                    max_ws = np.nanmax(node_ws_array, axis=0)  # Max over time for each node
                    avg_max_ws = np.nanmean(max_ws)
                    max_max_ws = np.nanmax(max_ws)
                    
                    # Store results in the dictionary
                    analysis_results[plan_name] = {
                        'Plan Number': plan_number,
                        'Simulation Duration (s)': sim_duration,
                        'Compute Time (hr)': compute_time,
                        'Compute Speed (hr/hr)': compute_speed,
                        'Average Max Pipe Flow (cfs)': avg_max_flow,
                        'Maximum Pipe Flow (cfs)': max_max_flow,
                        'Average Max Node Water Surface (ft)': avg_max_ws,
                        'Maximum Node Water Surface (ft)': max_max_ws,
                        'HDF Path': hdf_path
                    }
                    
                    print(f"  Simulation Duration: {sim_duration:.2f} seconds")
                    print(f"  Computation Time: {compute_time:.5f} hours")
                    print(f"  Computation Speed: {compute_speed:.2f} (simulation hours/compute hours)")
                    print(f"  Average Max Pipe Flow: {avg_max_flow:.2f} cfs")
                    print(f"  Maximum Pipe Flow: {max_max_flow:.2f} cfs")
                    print(f"  Average Max Node Water Surface: {avg_max_ws:.2f} ft")
                    print(f"  Maximum Node Water Surface: {max_max_ws:.2f} ft")
                    
                except Exception as e:
                    print(f"  Error analyzing pipe network data: {str(e)}")
                    analysis_results[plan_name] = {
                        'Plan Number': plan_number,
                        'Simulation Duration (s)': sim_duration,
                        'Compute Time (hr)': compute_time,
                        'Compute Speed (hr/hr)': compute_speed,
                        'Average Max Pipe Flow (cfs)': np.nan,
                        'Maximum Pipe Flow (cfs)': np.nan,
                        'Average Max Node Water Surface (ft)': np.nan,
                        'Maximum Node Water Surface (ft)': np.nan,
                        'HDF Path': hdf_path
                    }
            else:
                print("  No runtime data found.")
                analysis_results[plan_name] = {
                    'Plan Number': plan_number,
                    'Simulation Duration (s)': np.nan,
                    'Compute Time (hr)': np.nan,
                    'Compute Speed (hr/hr)': np.nan,
                    'Average Max Pipe Flow (cfs)': np.nan,
                    'Maximum Pipe Flow (cfs)': np.nan,
                    'Average Max Node Water Surface (ft)': np.nan,
                    'Maximum Node Water Surface (ft)': np.nan,
                    'HDF Path': hdf_path
                }
                
        except Exception as e:
            print(f"  Error analyzing plan {plan_number}: {str(e)}")
            analysis_results[plan_name] = {
                'Plan Number': plan_number,
                'Simulation Duration (s)': np.nan,
                'Compute Time (hr)': np.nan,
                'Compute Speed (hr/hr)': np.nan,
                'Average Max Pipe Flow (cfs)': np.nan,
                'Maximum Pipe Flow (cfs)': np.nan,
                'Average Max Node Water Surface (ft)': np.nan,
                'Maximum Node Water Surface (ft)': np.nan,
                'HDF Path': hdf_path
            }
    
    # Create a DataFrame from the analysis results
    analysis_df = pd.DataFrame.from_dict(analysis_results, orient='index')
    
    # Extract AEP years and handle NaN values
    analysis_df['AEP_Years'] = analysis_df.index.str.extract(r'(\d+)YR').astype(float)
    
    # Sort by AEP years, handling the base plan
    analysis_df = analysis_df.sort_values('AEP_Years', na_position='first')
    
    # Drop the temporary column used for sorting
    analysis_df = analysis_df.drop(columns=['AEP_Years'])
    
    print("\nAnalysis Results:")
    display.display(analysis_df)
    
    return analysis_df



In [ ]:
#---------------------------------------------------------------------
# Step 5: Analyze the results
#---------------------------------------------------------------------
print("\nStep 5: Analyzing the results...")

analysis_df = analyze_results(
    results=results,
    compute_folder=compute_folder,
    project=ras
)

In [46]:
def plot_results(analysis_df):
    """
    Plots the results from the analysis.
    """
    # Extract AEP values from the index (plan names), skipping non-AEP plans
    aep_values = []
    aep_data = pd.DataFrame()
    
    for name in analysis_df.index:
        if 'YR' in name:
            try:
                aep_year = int(name.split('YR')[0])
                aep_values.append(aep_year)
                aep_data = pd.concat([aep_data, analysis_df.loc[[name]]])
            except ValueError:
                continue
    
    if len(aep_values) == 0:
        print("No valid AEP plans found to plot")
        return
        
    # Create a figure with multiple subplots
    fig, axs = plt.subplots(2, 1, figsize=(14, 12))
    
    # Plot 1: Maximum Pipe Flow vs AEP
    axs[0].semilogx(aep_values, aep_data['Maximum Pipe Flow (cfs)'], 'o-', marker='o', markersize=8)
    axs[0].set_title('Maximum Pipe Flow vs Return Period', fontsize=16)
    axs[0].set_xlabel('Return Period (years)', fontsize=14)
    axs[0].set_ylabel('Maximum Pipe Flow (cfs)', fontsize=14)
    axs[0].grid(True)
    
    # Add data labels
    for i, txt in enumerate(aep_values):
        axs[0].annotate(f"{txt} yr", 
                      (aep_values[i], aep_data['Maximum Pipe Flow (cfs)'].iloc[i]),
                      textcoords="offset points", 
                      xytext=(0, 10), 
                      ha='center')
    
    # Plot 2: Maximum Node Water Surface vs AEP
    axs[1].semilogx(aep_values, aep_data['Maximum Node Water Surface (ft)'], 'o-', marker='s', markersize=8, color='green')
    axs[1].set_title('Maximum Node Water Surface vs Return Period', fontsize=16)
    axs[1].set_xlabel('Return Period (years)', fontsize=14)
    axs[1].set_ylabel('Maximum Node Water Surface (ft)', fontsize=14)
    axs[1].grid(True)
    
    # Add data labels
    for i, txt in enumerate(aep_values):
        axs[1].annotate(f"{txt} yr", 
                      (aep_values[i], aep_data['Maximum Node Water Surface (ft)'].iloc[i]),
                      textcoords="offset points", 
                      xytext=(0, 10), 
                      ha='center')
    
    plt.tight_layout()
    plt.show()
    # Plot time series for each return period
    try:
        plt.figure(figsize=(14, 8))
        
        # Create a color map for different return periods
        colors = plt.cm.viridis(np.linspace(0, 1, len(aep_values)))
        
        # Plot each return period
        for i, name in enumerate(aep_data.index):
            # Get HDF path for this return period
            hdf_path = aep_data.loc[name, 'HDF Path']
            
            # Get pipe network timeseries data
            node_ws = HdfPipe.get_pipe_network_timeseries(hdf_path, variable="Nodes/Water Surface")
            
            # Get data for location 61
            loc_61_ws = node_ws.sel(location=61)
            
            # Plot the time series
            plt.plot(loc_61_ws.time.values, loc_61_ws.values, 
                    label=f'{name}', 
                    color=colors[i],
                    linewidth=2)
        
        plt.title('Water Surface Elevation Time Series by Return Period - Location 61', fontsize=16)
        plt.xlabel('Time', fontsize=14)
        plt.ylabel('Water Surface Elevation (ft)', fontsize=14)
        plt.grid(True)
        plt.legend(fontsize=12)
        
        # Format x-axis dates
        plt.gcf().autofmt_xdate()
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Could not create detailed heatmap: {str(e)}")

In [47]:
#---------------------------------------------------------------------
# Step 6: Plot the results
#---------------------------------------------------------------------
print("\nStep 6: Plotting the results...")

plot_results(analysis_df)


Step 6: Plotting the results...


C:\Users\billk\AppData\Local\Temp\ipykernel_25136\3654224487.py:26: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o-" (-> marker='o'). The keyword argument will take precedence.
  axs[0].semilogx(aep_values, aep_data['Maximum Pipe Flow (cfs)'], 'o-', marker='o', markersize=8)
C:\Users\billk\AppData\Local\Temp\ipykernel_25136\3654224487.py:41: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o-" (-> marker='o'). The keyword argument will take precedence.
  axs[1].semilogx(aep_values, aep_data['Maximum Node Water Surface (ft)'], 'o-', marker='s', markersize=8, color='green')


<Figure size 1400x1200 with 2 Axes>

2025-03-20 11:53:52 - ras_commander.HdfPipe - INFO - Using HDF file from direct string path: c:\Users\billk\Desktop\RAS Commander Examples HOLDBACK\example_projects\Davis\compute_aep_parallel\DavisStormSystem.p01.hdf
2025-03-20 11:53:52 - ras_commander.HdfPipe - INFO - Final validated HDF file path: c:\Users\billk\Desktop\RAS Commander Examples HOLDBACK\example_projects\Davis\compute_aep_parallel\DavisStormSystem.p01.hdf
2025-03-20 11:53:52 - ras_commander.HdfPipe - INFO - Using HDF file from direct string path: c:\Users\billk\Desktop\RAS Commander Examples HOLDBACK\example_projects\Davis\compute_aep_parallel\DavisStormSystem.p03.hdf
2025-03-20 11:53:52 - ras_commander.HdfPipe - INFO - Final validated HDF file path: c:\Users\billk\Desktop\RAS Commander Examples HOLDBACK\example_projects\Davis\compute_aep_parallel\DavisStormSystem.p03.hdf
2025-03-20 11:53:52 - ras_commander.HdfPipe - INFO - Using HDF file from direct string path: c:\Users\billk\Desktop\RAS Commander Examples HOLDBACK\ex

<Figure size 1400x800 with 1 Axes>

In [ ]:


node_ws = HdfPipe.get_pipe_network_timeseries("04", variable="Nodes/Water Surface")




In [45]:
node_ws

xarray.DataArray with Unknown dimensions
[Full xarray output truncated during preprocessing]

In [ ]:
pipe_flow_ds = HdfPipe.get_pipe_network_timeseries("04", variable="Pipes/Pipe Flow DS")

In [40]:
pipe_flow_ds

xarray.DataArray with Unknown dimensions
[Full xarray output truncated during preprocessing]

## Plot time series for each event

# Plot time series for each AEP event at the same downstream location
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import MaxNLocator
import pandas as pd
from ras_commander import HdfResultsPlan

# Set up the figure with a larger size for better visibility
plt.figure(figsize=(14, 8))

# Set up color map for different ARI events
cmap = plt.cm.viridis
colors = [cmap(i) for i in np.linspace(0, 0.9, len(new_plan_numbers))]

# Dictionary to store all dataframes for later use
all_ts_data = {}

# Loop through each plan we created
for i, plan_number in enumerate(new_plan_numbers):
    try:
        # Get results HDF path for this plan
        results_path = RasPlan.get_results_path(plan_number, ras_object=ras)
        
        if not results_path.exists():
            print(f"No results found for Plan {plan_number}. Skipping.")
            continue
            
        # Extract the AEP value from the plan name
        plan_info = ras.plan_df.loc[int(plan_number)]
        plan_name = plan_info["Title"]
        aep_label = f"{plan_info['ShortID']} ({plan_name})"
        
        # Get reference point time series (using the same 'pipe_flow_ds' point)
        ref_ts = HdfResultsPlan.reference_points_timeseries_output(results_path)
        
        # Check if pipe_flow_ds exists in the results
        if 'pipe_flow_ds' not in ref_ts:
            print(f"Warning: 'pipe_flow_ds' not found in results for Plan {plan_number}")
            continue
        
        # Get the time series data for pipe_flow_ds
        ts_data = ref_ts['pipe_flow_ds']
        
        # Store data for potential further use
        all_ts_data[plan_number] = ts_data
        
        # Plot the stage (water surface elevation)
        if 'Stage' in ts_data.columns:
            plt.plot(ts_data.index, ts_data['Stage'], 
                     label=aep_label, 
                     color=colors[i], 
                     linewidth=2)
    except Exception as e:
        print(f"Error processing Plan {plan_number}: {e}")

# Add grid, legend and labels
plt.grid(True, alpha=0.3)
plt.legend(loc='best', fontsize=10)
plt.title('Water Surface Elevation Time Series at Downstream Point', fontsize=14)
plt.xlabel('Time', fontsize=12)
plt.ylabel('Stage (m)', fontsize=12)

# Format the x-axis to show dates nicely
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m-%d %H:%M'))
plt.xticks(rotation=45)

# Adjust y-axis to show reasonable number of ticks
plt.gca().yaxis.set_major_locator(MaxNLocator(nbins=10))

# Add a secondary plot with flow if it exists in the data
if all(('Flow' in df.columns) for df in all_ts_data.values() if not df.empty):
    ax2 = plt.gca().twinx()
    
    for i, (plan_number, ts_data) in enumerate(all_ts_data.items()):
        if 'Flow' in ts_data.columns:
            ax2.plot(ts_data.index, ts_data['Flow'], 
                     linestyle='--', 
                     color=colors[i], 
                     alpha=0.5)
    
    ax2.set_ylabel('Flow (m³/s)', color='gray', fontsize=12)
    ax2.tick_params(axis='y', labelcolor='gray')

plt.tight_layout()
plt.show()

# Create a table of peak values for each plan
print("\nPeak Values Summary:")
peak_data = []

for plan_number, ts_data in all_ts_data.items():
    plan_info = ras.plan_df.loc[int(plan_number)]
    aep_label = f"{plan_info['ShortID']} ({plan_info['Title']})"
    
    if 'Stage' in ts_data.columns:
        peak_stage = ts_data['Stage'].max()
        peak_time = ts_data['Stage'].idxmax()
        
        peak_flow = None
        if 'Flow' in ts_data.columns:
            peak_flow = ts_data['Flow'].max()
        
        peak_data.append({
            'Plan': aep_label,
            'Peak Stage (m)': round(peak_stage, 2),
            'Time of Peak': peak_time,
            'Peak Flow (m³/s)': round(peak_flow, 2) if peak_flow is not None else 'N/A'
        })

if peak_data:
    peak_df = pd.DataFrame(peak_data)
    display(peak_df)

## Conclusion

This notebook demonstrates a comprehensive workflow for automated AEP storm analysis using RAS-Commander. The key benefits of this approach include:

1. **Efficiency**: Automating repetitive tasks saves time and reduces errors
2. **Consistency**: Ensures consistent methodology across all return periods
3. **Parallel Execution**: Makes optimal use of computational resources
4. **Comprehensive Analysis**: Extracts and visualizes key metrics across return periods
5. **Reproducibility**: The entire workflow is documented and repeatable

This approach can be extended to include additional analyses, such as:

- Comparing different storm patterns (e.g., position of peak intensity)
- Analyzing climate change scenarios by adjusting precipitation depths
- Evaluating infrastructure improvements by comparing baseline and modified geometries
- Generating frequency curves for key hydraulic parameters

By leveraging the power of RAS-Commander, engineers can focus on interpreting results and making design decisions rather than managing model configurations.